In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# importa o knn
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# importa a decision tree
from sklearn import tree
# importa o random forest
from sklearn.ensemble import RandomForestClassifier
# importa o módulo de métricas
from sklearn import metrics
# importa a função auxiliar para fazer a divisao dos dados
from sklearn.model_selection import train_test_split
# importa o encoder para transformar as classes em texto em números
from sklearn.preprocessing import LabelEncoder
# importa o standard scaler para normalizar os dados numéricos
from sklearn.preprocessing import StandardScaler
# importa o minmax scaler para normalizar os dados numéricos
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU

In [149]:
dados = pd.read_csv('results.csv')

# dados.info()
# dados.head()

le = LabelEncoder()
scaler = MinMaxScaler()

# Time da casa ganhar = 2
# Empate = 1
# Time de fora ganhar = 0

# print(dados['result'][15])

le.fit(dados['home_team'])
dados['home_team'] = le.transform(dados['home_team'])
dados['away_team'] = le.transform(dados['away_team'])

le.fit(dados['result'])
dados['result'] = le.transform(dados['result'])

le.fit(dados['season'])
dados['season'] = le.transform(dados['season'])

# scaler.fit(dados[['home_goals']])
# dados['home_goals'] = scaler.transform(dados[['home_goals']])

# scaler.fit(dados[['away_goals']])
# dados['away_goals'] = scaler.transform(dados[['away_goals']])

dadosWin = dados.drop(['home_goals', 'away_goals', 'season'], axis=1)
dadosGoalsHome = dados.drop(['result', 'away_goals', 'season'], axis=1)
dadosGoalsAway = dados.drop(['result', 'home_goals', 'season'], axis=1)

# print(dados['result'][15])

dw_train_x, dw_test_x, dw_train_y, dw_test_y = train_test_split(dadosWin.drop(['result'], axis=1), dadosWin['result'], test_size=0.3, random_state=42, stratify=dadosWin['result'])
gh_train_x, gh_test_x, gh_train_y, gh_test_y = train_test_split(dadosGoalsHome.drop(['home_goals'], axis=1), dadosGoalsHome['home_goals'], test_size=0.3, random_state=42)
ga_train_x, ga_test_x, ga_train_y, ga_test_y = train_test_split(dadosGoalsAway.drop(['away_goals'], axis=1), dadosGoalsAway['away_goals'], test_size=0.3, random_state=42)
gh_train_x.head()

,home_team,away_team
1554,23,5
3441,12,2
1356,2,1
2908,33,14
1490,17,31


### Previsão de quem vai ganhar a partida

In [104]:
knn = KNeighborsClassifier(n_neighbors=20)

knn.fit(dw_train_x, dw_train_y)

pred_y = knn.predict(dw_test_x)

print("Acurácia:", metrics.accuracy_score(pred_y, dw_test_y))
print("F1-score:", metrics.f1_score(pred_y, dw_test_y, average='micro'))
print("Precision:", metrics.precision_score(pred_y, dw_test_y, average='micro'))
print("Recall:", metrics.recall_score(pred_y, dw_test_y, average='micro'))

Acurácia: 0.5007309941520468
F1-score: 0.5007309941520468
Precision: 0.5007309941520468
Recall: 0.5007309941520468


In [105]:
# Geramos a árvore de decisão utilizando gini como critério de cálculo de Information Gain
# clf = tree.DecisionTreeClassifier(criterion="gini", random_state=42)
clf = tree.DecisionTreeClassifier(criterion="gini", random_state=42, max_depth=7)
# clf = tree.DecisionTreeClassifier(criterion="log_loss", random_state=42)

#Treinamos o modelo utilizando treino X e treino Y
clf.fit(dw_train_x, dw_train_y)

# Predizemos o conteúdo do teste X
pred_y = clf.predict(dw_test_x)

print("Acurácia:", metrics.accuracy_score(pred_y, dw_test_y))
print("F1-score:", metrics.f1_score(pred_y, dw_test_y, average='micro'))
print("Precision:", metrics.precision_score(pred_y, dw_test_y, average='micro'))
print("Recall:", metrics.recall_score(pred_y, dw_test_y, average='micro'))

Acurácia: 0.489766081871345
F1-score: 0.489766081871345
Precision: 0.489766081871345
Recall: 0.489766081871345


In [106]:
# Crie o modelo da rede neural
model = Sequential()

# Adicione a primeira camada oculta com ativação ReLU e 10 neurônios
model.add(Dense(5, input_dim=2, activation='relu'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(Dense(5, activation='relu'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(Dense(2, activation='relu'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(Dense(30, activation='relu'))
# model.add(LeakyReLU(alpha=0.1))
# model.add(Dense(30, activation='relu'))
# model.add(LeakyReLU(alpha=0.1))

# # Adicione a camada de saída com ativação Softmax para classificação binária
model.add(Dense(1, activation='softmax'))

# Compile o modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treine o modelo
model.fit(dw_train_x, dw_train_y, epochs=500, batch_size=300)#, validation_data=(dw_test_x, dw_test_y))

# Avalie o modelo
loss, accuracy = model.evaluate(dw_train_x, dw_train_y)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/500
11/11 [==============================] - 1s 2ms/step - loss: -2.8768 - accuracy: 0.2553
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: -3.0579 - accuracy: 0.2553
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: -3.2377 - accuracy: 0.2553
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: -3.4100 - accuracy: 0.2553
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: -3.5818 - accuracy: 0.2553
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: -3.7747 - accuracy: 0.2553
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: -3.9490 - accuracy: 0.2553
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: -4.1471 - accuracy: 0.2553
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: -4.3457 - accuracy: 0.2553
Epoch 10/500
11/11 [==============================] - 0s 2ms/step - loss: -4.5442 - accuracy: 0.2553

In [107]:
pred = knn.predict([[20, 23]])
print(pred)

pred2 = model.predict([[20, 23]])
print(pred2)

pred3 = clf.predict([[20, 23]])
print(pred3)

[2]
1/1 [==============================] - 0s 65ms/step
[[1.]]
[2]


c:\Users\BeatsCode\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\BeatsCode\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


### Previsão de gols da partida

In [157]:
knnh = KNeighborsRegressor(n_neighbors=20)

knnh.fit(gh_train_x, gh_train_y)

pred_y = knnh.predict(gh_test_x)

mse = metrics.mean_squared_error(pred_y, gh_test_y)
rmse = np.sqrt(mse)
mae = metrics.mean_absolute_error(pred_y, gh_test_y)
r2 = metrics.r2_score(pred_y, gh_test_y)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)
print("Média:", dados['home_goals'].mean())
print()

knna = KNeighborsRegressor(n_neighbors=20)

knna.fit(ga_train_x, ga_train_y)

pred_y = knna.predict(ga_test_x)

mse = metrics.mean_squared_error(pred_y, ga_test_y)
rmse = np.sqrt(mse)
mae = metrics.mean_absolute_error(pred_y, ga_test_y)
r2 = metrics.r2_score(pred_y, ga_test_y)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)
print("Média:", dados['away_goals'].mean())
print()

predh = knnh.predict([[20, 23]])
preda = knna.predict([[20, 23]])
print(predh, preda)

Mean Squared Error: 1.6344773391812866
Root Mean Squared Error: 1.2784667923654829
Mean Absolute Error: 1.0009502923976608
R-squared: -7.330175484883842
Média: 1.5432017543859649

Mean Squared Error: 1.3275274122807017
Root Mean Squared Error: 1.1521837580354541
Mean Absolute Error: 0.9067616959064327
R-squared: -8.55034759778039
Média: 1.1442982456140351

[2.65] [0.85]


c:\Users\BeatsCode\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
c:\Users\BeatsCode\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
